In [1]:
from keras.models import Sequential  
from keras.layers.core import Dense, Activation, Dropout, TimeDistributedDense,Flatten
from keras.layers.recurrent import LSTM
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD,Adadelta,RMSprop
from keras.layers.embeddings import Embedding
from keras.layers import Merge
import pandas as pd  
from random import random
import numpy as np
import talib as ta
from datetime import datetime
from pandas.io import data, wb
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
import urllib2
import pytz
from bs4 import BeautifulSoup
import copy
import csv
import urllib2
import datetime as dt
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from sklearn.cross_validation import train_test_split

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 4007)
/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/pandas/io/data.py:33: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
/home/ai2-jedi/anaconda2/lib/python2.7/site-packages/pandas/io/wb.py:19: FutureWarning: 
The pandas.io.wb module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
/home/ai2-jedi/anaconda2/lib/py

In [2]:
dset = []
with open('User_085_Mode_Transport_Medium_Cluster_v2.csv','rb') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        dset.append([row[3],row[4],row[5],row[7],row[8],row[9],row[6]])

In [3]:
clusters = {}
dset = dset[1:]
for row in dset:
    #print row[6]
    if row[6] not in clusters.keys():
        clusters[row[6]] = 1
    else:
        clusters[row[6]] = clusters[row[6]] + 1
print clusters

{'5.0': 46, '1.0': 1073, '4.0': 79, '800.0': 11, '0.0': 12373, '3.0': 11390, '6.0': 28}


In [4]:
for row in range(len(dset)):
    val = dset[row][6]
    if val != '0.0' and dset[row][6] != '3.0':
        dset[row][6] = '2.0'
        
for row in range(len(dset)):
    if dset[row][6] == '3.0':
        dset[row][6] = '1.0'

In [5]:
clusters = {}
for row in dset:
    if row[6] not in clusters.keys():
        clusters[row[6]] = 1
    else:
        clusters[row[6]] = clusters[row[6]] + 1
print clusters

{'0.0': 12373, '2.0': 1237, '1.0': 11390}


In [6]:
print dset[0]
print dset[1]
#dset = dset[1:]
print (len(dset))
modes = {}
for row in dset:
    if row[3] not in modes.keys():
        modes[row[3]] = 1
    else:
        modes[row[3]] = modes[row[3]] + 1
print modes

['39.9762', '116.33032', '2.48579225103', 'walk', '13', '56', '0.0']
['39.97619', '116.33032', '0.32437826823', 'walk', '13', '56', '0.0']
25000
{'taxi': 380, 'bus': 5568, 'walk': 19052}


In [7]:
classes = {}
for (i,j) in enumerate(modes.keys()):
    classes[j] = i
print (classes)

{'taxi': 0, 'bus': 1, 'walk': 2}


In [8]:
for row in range(len(dset)):
    dset[row][3] = int(classes[dset[row][3]])
    dset[row][0] = float(dset[row][0])
    dset[row][1] = float(dset[row][1])
    dset[row][2] = float(dset[row][2])
    dset[row][4] = float(dset[row][4])
    dset[row][5] = float(dset[row][5])
    dset[row][6] = float(dset[row][6])
print(dset[0])

[39.9762, 116.33032, 2.48579225103, 2, 13.0, 56.0, 0.0]


In [20]:
SEQ_LEN = 360 #45 Mins
N_HIDDEN = 100
NUM_CLASSES = 3
NUM_ATTR = 6
LOOK_FORWARD = 360 #15

def createSeqData(data, n_prev = SEQ_LEN):
    docX, docY = [], []
    num_sequences = (len(data)-n_prev - LOOK_FORWARD)
    for i in range(num_sequences):
        x = (data.iloc[i:i+n_prev,:NUM_ATTR].as_matrix())
        y = (data.iloc[i+n_prev + LOOK_FORWARD,NUM_ATTR:].as_matrix())        
        docX.append(x)
        docY.append(y)
    alsX = np.array(docX)
    alsY = np.array(docY)
    return alsX, alsY


def splitDset(dset, ratio):
    index = int(round(ratio*len(dset)))
    #print (index)
    train_dset = dset[:index]
    test_dset = dset[index:]
    return train_dset, test_dset

### sklearn split dataset

In [21]:
df = pd.DataFrame(np.asarray(dset))
X, y = createSeqData(df)
a_train, a_test, b_train, b_test = train_test_split(X, y, test_size=0.33, random_state=23)
b_train = b_train.astype(int)
b_train = to_categorical(b_train)
print(a_train.shape)
print(b_train.shape)

(16267, 360, 6)
(16267, 3)


In [22]:
print (a_train[0])
print (b_train[0])

[[  39.9396      116.30452       4.49063971    2.            9.           16.        ]
 [  39.93951     116.30451       2.51485211    2.            9.           16.        ]
 [  39.93946     116.3045        3.37787796    2.            9.           16.        ]
 ..., 
 [  39.97012     116.3006        0.38101329    2.            9.           47.        ]
 [  39.97012     116.30059       0.            2.            9.           47.        ]
 [  39.97012     116.30059       0.38101268    2.            9.           48.        ]]
[ 1.  0.  0.]


### Manually Split Dataset

In [23]:
model_intra = Sequential() 
model_intra.add(LSTM(N_HIDDEN, return_sequences=True, activation='tanh', input_shape=(SEQ_LEN, NUM_ATTR)))
model_intra.add(LSTM(N_HIDDEN, return_sequences=False, activation='tanh'))
model_intra.add(Dense(3, activation='softmax'))
model_intra.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [25]:
model_intra.fit(a_train, b_train, batch_size=250, nb_epoch=15, validation_split=0.20)

Train on 13013 samples, validate on 3254 samples
Epoch 1/15
13013/13013 [==============================] - 37s - loss: 0.7655 - acc: 0.5990 - val_loss: 0.6542 - val_acc: 0.7099
Epoch 2/15
13013/13013 [==============================] - 37s - loss: 0.6257 - acc: 0.7187 - val_loss: 0.5656 - val_acc: 0.7566
Epoch 3/15
13013/13013 [==============================] - 37s - loss: 0.5494 - acc: 0.7543 - val_loss: 0.6308 - val_acc: 0.6853
Epoch 4/15
13013/13013 [==============================] - 37s - loss: 0.5012 - acc: 0.7893 - val_loss: 0.4394 - val_acc: 0.8236
Epoch 5/15
13013/13013 [==============================] - 37s - loss: 0.4864 - acc: 0.7981 - val_loss: 0.6110 - val_acc: 0.6998
Epoch 6/15
13013/13013 [==============================] - 37s - loss: 0.4763 - acc: 0.7990 - val_loss: 0.3391 - val_acc: 0.8682
Epoch 7/15
13013/13013 [==============================] - 37s - loss: 0.4185 - acc: 0.8349 - val_loss: 0.3222 - val_acc: 0.8672
Epoch 8/15
13013/13013 [==============================]

# Test

In [26]:
print(len(a_test))
print(len(b_test))

8013
8013


In [27]:
Y_pred = model_intra.predict(a_test)

In [28]:
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i].argmax() == b_test[i]:
        count = count + 1
        
acc = count/float(len(b_test))
print(acc)

0.836515662049


In [29]:
cluster1_total = 0
cluster1_correct = 0
cluster2_count = 0
cluster2_correct = 0
cluster3_total = 0
cluster3_correct = 0


for i in range(len(Y_pred)):

        
    if b_test[i] == 0:
        cluster1_total = cluster1_total + 1
        
        if Y_pred[i].argmax() == b_test[i]:
            cluster1_correct = cluster1_correct + 1
            
    if b_test[i] == 1:
        cluster2_count = cluster2_count + 1
        
        if Y_pred[i].argmax() == b_test[i]:
            cluster2_correct = cluster2_correct + 1
            
    if b_test[i] == 2:
        cluster3_total = cluster3_total + 1
        
        if Y_pred[i].argmax() == b_test[i]:
            cluster3_correct = cluster3_correct + 1
            

            
            
            
            
cluster1_acc = cluster1_correct/float(cluster1_total)
cluster2_acc = cluster2_correct/float(cluster2_count)
print ('Cluster 1:')
print(cluster1_correct, cluster1_total, cluster1_acc)

print ('Cluster 2:')
print(cluster2_correct, cluster2_count, cluster2_acc)

cluster3_acc = cluster3_correct/float(cluster3_total)
print ('Cluster 3(Other):')
print(cluster3_correct, cluster3_total, cluster3_acc)


print ('Total:')
total_correct = cluster1_correct + cluster2_correct + cluster3_correct
acc = total_correct/float(len(Y_pred))
print(acc)

Cluster 1:
(3007, 3997, 0.7523142356767576)
Cluster 2:
(3417, 3726, 0.9170692431561996)
Cluster 3(Other):
(279, 290, 0.9620689655172414)
Total:
0.836515662049
